In [43]:
# %pip install networkx scipy argparse

In [44]:
"""
GEXF -> HTML interativo (D3.js + Tabulator) para a rede de coautoria entre PPGs (Fiocruz)

- Lê o arquivo .gexf informado na linha de comando.
- Calcula layout com spring_layout (seed=42, k=0.5, iterations=400, scale=0.7) e grava x,y nos nós.
- Cor do vértice: proporcional a "Conexões" (menor→maior) via paleta de 10 cores.
- Tamanho do vértice: constante para todos.
- Gera um arquivo .html (mesmo prefixo do .gexf) com:
  * Grafo D3 (zoom/drag, busca, ocultar/mostrar rótulos, limpar seleção)
  * Tabela Tabulator de vértices (id, label, Publicações_totais, Publicações_em_coautoria, Conexões, Proporção_da_coautoria_Fiocruz, cor)
  * Tabela Tabulator de arestas (par label-label, peso, nº de vizinhos em comum)

Requisitos: networkx (>=2.8)
"""

import argparse
import json
import os
import sys
import math
import html
import networkx as nx
import scipy


In [45]:
PALETTE = [
    "#5e4fa2", "#3288bd", "#66c2a5", "#abdda4", "#e6f598",
    "#fee08b", "#fdae61", "#f46d43", "#d53e4f", "#9e0142"
]

In [46]:
def parse_args():
    ap = argparse.ArgumentParser(description="GEXF -> HTML interativo (D3 + Tabulator) para rede de coautoria PPGs Fiocruz")
    ap.add_argument("gexf_path", help="Caminho do arquivo .gexf de entrada")
    ap.add_argument("--seed", type=int, default=42)
    ap.add_argument("--k", type=float, default=0.5)
    ap.add_argument("--iterations", type=int, default=400)
    ap.add_argument("--scale", type=float, default=0.7)
    ap.add_argument("--node_radius", type=float, default=8.0, help="Raio (constante) dos nós no D3")
    return ap.parse_args()

def linear_color(value, vmin, vmax, palette):
    if value is None or math.isnan(float(value)):
        return palette[0]
    if vmax <= vmin:
        return palette[0]
    t = (float(value) - float(vmin)) / (float(vmax) - float(vmin))
    t = 0.0 if t < 0 else (1.0 if t > 1 else t)
    idx = int(round(t * (len(palette) - 1)))
    if idx < 0: idx = 0
    if idx >= len(palette): idx = len(palette) - 1
    return palette[idx]

# def build_graph(gexf_path, seed, k, iterations, scale):
#     # Lê o grafo (NetworkX garante leitura de atributos do GEXF)
#     G = nx.read_gexf(gexf_path)

#     # Se não houver posições, calcula com spring_layout
#     #pos = nx.spring_layout(G, seed=seed, k=k, iterations=iterations, scale=scale)
#     pos = nx.kamada_kawai_layout(G, scale=1.0)

#     # Extrai "Conexões" (fall-back para grau de grafo se não existir)
#     conexoes_values = []
#     for n in G.nodes():
#         conex = None
#         data = G.nodes[n]
#         # ‘Conexões’ pode vir como string; padroniza para int
#         if "Conexões" in data:
#             try:
#                 conex = int(data["Conexões"])
#             except Exception:
#                 try:
#                     conex = int(float(str(data["Conexões"]).replace(",", ".")))
#                 except Exception:
#                     conex = None
#         if conex is None:
#             conex = int(G.degree[n])  # fallback
#             G.nodes[n]["Conexões"] = conex
#         conexoes_values.append(conex)

#     vmin = min(conexoes_values) if conexoes_values else 0
#     vmax = max(conexoes_values) if conexoes_values else 1

#     # Atribui x,y e cor (de acordo com Conexões)
#     for n, (x, y) in pos.items():
#         G.nodes[n]["x"] = float(x)
#         G.nodes[n]["y"] = float(y)
#         conex = G.nodes[n].get("Conexões", 0)
#         G.nodes[n]["color"] = linear_color(conex, vmin, vmax, PALETTE)

#     # Peso da aresta (fallback = 1)
#     for u, v, d in G.edges(data=True):
#         w = d.get("weight", 1)
#         try:
#             w = float(str(w).replace(",", "."))
#         except Exception:
#             w = 1.0
#         d["weight"] = w

#     return G

def build_graph(gexf_path, seed, k, iterations, scale):
    # Lê o grafo
    G = nx.read_gexf(gexf_path)

    # Calcula layout (mantendo o Kamada-Kawai que já estava no seu código)
    # pos = nx.kamada_kawai_layout(G, scale=1.0)
    pos = nx.spring_layout(G, seed=42, k=0.15, iterations=150, scale=1000)

    # Extrai valores para definir a escala de cores.
    # Vamos usar 'total_publications' como métrica principal para a cor.
    # Se preferir usar centralidade, troque por "centralidade_grau".
    metric_key = "total_publications"
    metric_values = []

    for n in G.nodes():
        data = G.nodes[n]
        val = data.get(metric_key, 0)
        try:
            val = float(val)
        except:
            val = 0.0
        metric_values.append(val)
        # Garante que o dado esteja limpo no nó
        G.nodes[n][metric_key] = val

    vmin = min(metric_values) if metric_values else 0
    vmax = max(metric_values) if metric_values else 1

    # Atribui x,y e cor
    for n, (x, y) in pos.items():
        G.nodes[n]["x"] = float(x)
        G.nodes[n]["y"] = float(y)
        
        # Pega o valor da métrica para colorir
        val = G.nodes[n].get(metric_key, 0)
        G.nodes[n]["color"] = linear_color(val, vmin, vmax, PALETTE)

    # Tratamento de peso das arestas (mantido igual)
    for u, v, d in G.edges(data=True):
        w = d.get("weight", 1)
        try:
            w = float(str(w).replace(",", "."))
        except Exception:
            w = 1.0
        d["weight"] = w

    return G

In [47]:
def compute_common_neighbors_table(G, id_to_label):
    """
    Retorna lista de linhas para a tabela de arestas:
    { "aresta": "LabelA, LabelB", "colabs": weight, "comuns": n_comum }
    (par único por ordem alfabética de ID)
    """
    # vizinhanças por nó (IDs em string)
    neigh = {str(n): set(map(str, G.neighbors(n))) for n in G.nodes()}
    seen = set()
    rows = []
    for u, v, d in G.edges(data=True):
        a, b = sorted([str(u), str(v)])
        key = a + "||" + b
        if key in seen:
            continue
        seen.add(key)
        w = float(d.get("weight", 1) or 1)
        comuns = len((neigh.get(a, set()) & neigh.get(b, set())) - {a, b})
        la = id_to_label.get(a, a)
        lb = id_to_label.get(b, b)
        aresta_str = ", ".join(sorted([la, lb]))
        rows.append({
            "aresta": aresta_str,
            "colabs": w,
            "comuns": comuns
        })
    # ordena: colabs desc, comuns desc, aresta asc
    rows.sort(key=lambda r: (-r["colabs"], -r["comuns"], r["aresta"]))
    return rows


In [48]:
def graph_to_embeddable_json(G, node_radius_const=8.0):
    nodes, id_to_label = [], {}
    for n, data in G.nodes(data=True):
        nid, label = str(n), str(data.get("label", n))
        id_to_label[nid] = label

    for n, data in G.nodes(data=True):
        nid, label = str(n), id_to_label[nid]
        
        # --- Tratamento robusto do campo similares ---
        similares_str = str(data.get("similares_node2vec", "[]"))
        try:
            # Tenta converter a string JSON do GEXF em lista Python real
            # Isso evita problemas de aspas (" vs ') no JavaScript
            import json
            similares_data = json.loads(similares_str.replace("'", '"'))
        except:
            similares_data = [] # Se falhar, deixa vazio para não quebrar o site
        
        # --- Tratamento de valores nulos/NaN ---
        def safe_float(val):
            try:
                f = float(val)
                return 0.0 if math.isnan(f) else f
            except:
                return 0.0

        nodes.append({
            "id": nid,
            "label": label,
            "x": safe_float(data.get("x", 0)),
            "y": safe_float(data.get("y", 0)),
            "r": node_radius_const,
            "color": data.get("color", "#1f77b4"),
            # Novos atributos
            "total_publications": int(safe_float(data.get("total_publications", 0))),
            "score": safe_float(data.get("score", 0)),
            "centralidade_grau": safe_float(data.get("centralidade_grau", 0)),
            "pagerank": safe_float(data.get("pagerank", 0)),
            "hub_score": safe_float(data.get("hub_score", 0)),
            "similares": similares_data  # Passa como objeto, não string
        })

    links = [
        {"source": str(u), "target": str(v), "weight": float(d.get("weight", 1))}
        for u, v, d in G.edges(data=True)
    ]

    edge_rows = compute_common_neighbors_table(G, id_to_label)
    
    vertex_rows = [
        {
            "id": n["id"],
            "nome": n["label"],
            "total_publications": n["total_publications"],
            "score": n["score"],
            "centralidade_grau": n["centralidade_grau"],
            "pagerank": n["pagerank"],
            "cor": n["color"]
        }
        for n in nodes
    ]

    return {
        "graph": {"nodes": nodes, "links": links},
        "vertex_table": vertex_rows,
        "edge_table": edge_rows
    }

# def graph_to_embeddable_json(G, node_radius_const=8.0):
#     nodes, id_to_label = [], {}
#     for n, data in G.nodes(data=True):
#         nid, label = str(n), str(data.get("label", n))
#         id_to_label[nid] = label

#     for n, data in G.nodes(data=True):
#         nid, label = str(n), id_to_label[nid]
#         nodes.append({
#             "id": nid,
#             "label": label,
#             "x": float(data.get("x", 0)),
#             "y": float(data.get("y", 0)),
#             "r": node_radius_const,
#             "color": data.get("color", "#1f77b4"),
#             "Publicações_totais": int(data.get("Publicações_totais", 0) or 0),
#             "Publicações_em_coautoria": int(data.get("Publicações_em_coautoria", 0) or 0),
#             "Conexões": int(data.get("Conexões", 0) or 0),
#             "Proporção_da_coautoria_Fiocruz": data.get("Proporção_da_coautoria_Fiocruz")
#         })

#     links = [
#         {"source": str(u), "target": str(v), "weight": float(d.get("weight", 1))}
#         for u, v, d in G.edges(data=True)
#     ]

#     edge_rows = compute_common_neighbors_table(G, id_to_label)
#     vertex_rows = [
#         {
#             "id": n["id"],
#             "nome": n["label"],
#             "Publicações_totais": n["Publicações_totais"],
#             "Publicações_em_coautoria": n["Publicações_em_coautoria"],
#             "Conexões": n["Conexões"],
#             "Proporção_da_coautoria_Fiocruz": n["Proporção_da_coautoria_Fiocruz"],
#             "cor": n["color"]
#         }
#         for n in nodes
#     ]

#     return {
#         "graph": {"nodes": nodes, "links": links},
#         "vertex_table": vertex_rows,
#         "edge_table": edge_rows
#     }


In [49]:
def html_template(graph_json_str, vertex_table_str, edge_table_str):
    D3_CDN = "https://cdnjs.cloudflare.com/ajax/libs/d3/7.8.5/d3.min.js"
    TABULATOR_CSS = "https://cdnjs.cloudflare.com/ajax/libs/tabulator/5.5.0/css/tabulator.min.css"
    TABULATOR_JS  = "https://cdnjs.cloudflare.com/ajax/libs/tabulator/5.5.0/js/tabulator.min.js"
    XLSX_JS       = "https://cdnjs.cloudflare.com/ajax/libs/xlsx/0.18.5/xlsx.full.min.js"

    html_str = """
<!doctype html>
<html lang="pt-BR">
<head>
<meta charset="utf-8">
<title>Rede de Coautoria - Dados Cientométricos</title>
<meta name="viewport" content="width=device-width, initial-scale=1">
<link rel="stylesheet" href="__TABULATOR_CSS__">
<style>
  body { margin: 16px; font: 14px/1.45 system-ui, -apple-system, sans-serif; color:#111; }
  h1,h2,h3,h4 { margin: 0.2rem 0 0.7rem; }
  .viz-wrap { display:flex; gap:1rem; align-items: stretch; }
  #viz-container { flex:3; min-height: 600px; height: 75vh; border:1px solid #ccc; position:relative; background: #fff;}
  #viz { width:100%; height:100%; display:block; }
  #info-pane { flex:1; max-height: 75vh; overflow:auto; border:1px solid #ddd; padding:1rem; background:#f9f9f9; }
  .toolbar { display:flex; gap:0.5rem; align-items:center; margin-bottom:0.5rem; flex-wrap: wrap; }
  .toolbar input[type="text"] { flex:1; min-width: 200px; padding:0.5rem; }
  .toolbar button { padding:0.5rem 0.75rem; cursor:pointer; }
  .legend { display:flex; align-items:center; gap:.5rem; flex-wrap:wrap; font-size:12px; margin-right: 1rem;}
  .legend .item { display:inline-flex; align-items:center; gap:.3rem; margin-right:.5rem; }
  .legend .swatch { width:12px; height:12px; display:inline-block; }
  
  .node circle { cursor:pointer; stroke:#fff; stroke-width:1px; }
  .node:active circle { cursor:grabbing; }
  .node.selected circle { stroke:#000; stroke-width:2px; }
  .dimmed { opacity:0.1; }
  .label { font-size: 10px; pointer-events:none; text-shadow: 0 1px 0 #fff, 1px 0 0 #fff, 0 -1px 0 #fff, -1px 0 0 #fff; }
  .edge { stroke:#999; stroke-opacity:0.6; fill:none; }
  .edge.highlight { stroke:#555; stroke-opacity: 0.9; }
  
  /* Classe usada para ocultar os rótulos */
  .hidden { display:none !important; }
</style>
</head>
<body>

<h2>Rede de Coautoria e Indicadores</h2>
<p>Nós coloridos por <strong>Total de Publicações</strong>. Clique para ver métricas e similares.</p>

<div class="viz-wrap">
  <div style="flex:3; display:flex; flex-direction:column;">
    <div class="toolbar">
      <div class="legend" id="legend"></div>
      <button id="btnToggleLabels">Ocultar Rótulos</button>
      <input id="searchBox" type="text" placeholder="Buscar Nome..." />
      <button id="btnSearch">Buscar</button>
      <button id="btnClear">Limpar</button>
    </div>
    <div id="viz-container"><svg id="viz"></svg></div>
  </div>
  <div id="info-pane">
    <h4>Detalhes</h4>
    <p style="color:#666;">Clique em um nó para ver Score, Centralidade e Autores Similares.</p>
  </div>
</div>

<hr>
<h3>Dados dos Pesquisadores</h3>
<div id="tabela-vertices"></div>

<script src="__D3_CDN__"></script>
<script src="__TABULATOR_JS__"></script>
<script src="__XLSX_JS__"></script>

<script>
  var GRAPH_DATA = __GRAPH_JSON__;
  var DATA_VERT  = __VERTEX_TABLE_JSON__;
  
  // --- Config D3 ---
  var svg = d3.select("#viz"), g = svg.append("g");
  var zoom = d3.zoom().scaleExtent([0.1, 8]).on("zoom", e => g.attr("transform", e.transform));
  svg.call(zoom);
  
  // Camadas
  var linkG = g.append("g"), nodeG = g.append("g"), labelG = g.append("g");

  // Links
  var links = GRAPH_DATA.links.map(d => Object.create(d));
  var nodes = GRAPH_DATA.nodes.map(d => Object.create(d));

  // Desenho Arestas
  var link = linkG.selectAll(".edge")
    .data(links).join("path")
    .attr("class", "edge")
    .attr("stroke-width", d => Math.sqrt(d.weight || 1));

  // Desenho Nós
  var node = nodeG.selectAll(".node")
    .data(nodes, d => d.id).join("g").attr("class", "node")
    .attr("transform", d => `translate(${d.x},${d.y})`);
    
  node.append("circle")
    .attr("r", d => d.r)
    .attr("fill", d => d.color);

  // Rótulos
  var label = labelG.selectAll(".label")
    .data(nodes).join("text").attr("class", "label")
    .attr("text-anchor", "middle")
    .attr("dy", -10)
    .attr("x", d => d.x).attr("y", d => d.y)
    .text(d => d.label);

  // --- Lógica do Botão Rótulos (CORRIGIDO AQUI) ---
  var labelsVisible = true;
  document.getElementById("btnToggleLabels").onclick = function() {
      labelsVisible = !labelsVisible;
      // Alterna a classe 'hidden' no grupo de rótulos
      labelG.classed("hidden", !labelsVisible);
      // Atualiza o texto do botão
      this.textContent = labelsVisible ? "Ocultar Rótulos" : "Mostrar Rótulos";
  };

  // --- Lógica de Seleção ---
  var selectedId = null;
  function getNeighbors(id) {
    const s = new Set([id]);
    links.forEach(l => {
      const src = l.source.id || l.source;
      const tgt = l.target.id || l.target;
      if(src === id) s.add(tgt);
      if(tgt === id) s.add(src);
    });
    return s;
  }

  function updateHighlight() {
    if(!selectedId) {
      d3.selectAll(".dimmed").classed("dimmed", false);
      return;
    }
    const n = getNeighbors(selectedId);
    node.classed("dimmed", d => !n.has(d.id));
    label.classed("dimmed", d => !n.has(d.id));
    link.classed("dimmed", l => {
        const s = l.source.id || l.source;
        const t = l.target.id || l.target;
        return !(s === selectedId || t === selectedId);
    });
  }

  // --- Painel Lateral ---
  function showInfo(d) {
    let html = `<h3>${d.label}</h3>`;
    html += `<ul style="padding-left:1rem; line-height:1.6;">`;
    html += `<li><strong>Publicações:</strong> ${d.total_publications}</li>`;
    html += `<li><strong>Score:</strong> ${d.score.toFixed(4)}</li>`;
    html += `<li><strong>Centralidade Grau:</strong> ${d.centralidade_grau.toFixed(5)}</li>`;
    html += `<li><strong>PageRank:</strong> ${d.pagerank.toFixed(5)}</li>`;
    html += `</ul>`;

    if(d.similares && d.similares.length > 0) {
        html += `<hr><strong>Similares (Node2Vec):</strong><br><small>`;
        d.similares.slice(0, 8).forEach(s => {
            let simVal = typeof s.similaridade === 'number' ? (s.similaridade*100).toFixed(1) : "?";
            html += `<div style="margin-top:4px;">• ${s.nome} <span style="color:#666">(${simVal}%)</span></div>`;
        });
        html += `</small>`;
    }
    document.getElementById("info-pane").innerHTML = html;
  }

  node.on("click", (e, d) => {
    e.stopPropagation();
    selectedId = d.id;
    updateHighlight();
    showInfo(d);
  });

  svg.on("click", () => { selectedId = null; updateHighlight(); });

  // --- Busca ---
  function search(val) {
    if(!val) return;
    val = val.toLowerCase();
    const hit = nodes.find(n => n.label.toLowerCase().includes(val));
    if(hit) {
       selectedId = hit.id;
       updateHighlight();
       showInfo(hit);
       const t = d3.zoomIdentity.translate(svg.attr("width")/2 - hit.x, svg.attr("height")/2 - hit.y);
       svg.transition().duration(750).call(zoom.transform, t);
    }
  }
  document.getElementById("btnSearch").onclick = () => search(document.getElementById("searchBox").value);
  document.getElementById("btnClear").onclick = () => { selectedId = null; updateHighlight(); };

  // --- Tabela Tabulator ---
  new Tabulator("#tabela-vertices", {
    data: DATA_VERT,
    layout: "fitColumns",
    height: "400px",
    pagination: "local",
    paginationSize: 20,
    columns: [
      {title:"Nome", field:"nome", headerFilter:"input"},
      {title:"Pubs", field:"total_publications", sorter:"number", width:80},
      {title:"Score", field:"score", sorter:"number", width:100},
      {title:"PageRank", field:"pagerank", sorter:"number", width:100},
    ]
  });
  
  // --- Inicialização de Layout ---
  function updateEdgePositions() {
      const nodeMap = new Map(nodes.map(n => [n.id, n]));
      link.attr("d", d => {
          const s = nodeMap.get(d.source) || d.source;
          const t = nodeMap.get(d.target) || d.target;
          return `M${s.x},${s.y} L${t.x},${t.y}`;
      });
  }
  updateEdgePositions();

  const bounds = g.node().getBBox();
  const parent = document.getElementById("viz-container").getBoundingClientRect();
  const scale = Math.min(parent.width / bounds.width, parent.height / bounds.height) * 0.85;
  const tx = (parent.width - bounds.width * scale) / 2 - bounds.x * scale;
  const ty = (parent.height - bounds.height * scale) / 2 - bounds.y * scale;
  svg.call(zoom.transform, d3.zoomIdentity.translate(tx, ty).scale(scale));

</script>
</body>
</html>
"""
    html_str = (html_str
        .replace("__TABULATOR_CSS__", TABULATOR_CSS)
        .replace("__D3_CDN__", D3_CDN)
        .replace("__TABULATOR_JS__", TABULATOR_JS)
        .replace("__XLSX_JS__", XLSX_JS)
        .replace("__GRAPH_JSON__", graph_json_str)
        .replace("__VERTEX_TABLE_JSON__", vertex_table_str)
    )
    return html_str

# def html_template(graph_json_str, vertex_table_str, edge_table_str):
#     D3_CDN = "d3.min.js"
#     TABULATOR_CSS = "tabulator_site.min.css"
#     TABULATOR_JS  = "tabulator.min.js"
#     XLSX_JS       = "xlsx.full.min.js"

#     # Usar placeholders únicos (sem chaves) para evitar conflitos:
#     html_str = """
# <!doctype html>
# <html lang="pt-BR">
# <head>
# <meta charset="utf-8">
# <title>Rede de Coautoria entre PPGs (Fiocruz)</title>
# <meta name="viewport" content="width=device-width, initial-scale=1">

# <link rel="stylesheet" href::__TABULATOR_CSS__>
# <style>
#   body { margin: 16px; font: 14px/1.45 system-ui, -apple-system, "Segoe UI", Roboto, Ubuntu, sans-serif; color:#111; }
#   h1,h2,h3,h4 { margin: 0.2rem 0 0.7rem; }
#   .viz-wrap { display:flex; gap:1rem; align-items: stretch; }
#   #viz-container { flex:3; min-height: 620px; height: 70vh; border:1px solid #ccc; position:relative; }
#   #viz { width:100%; height:100%; display:block; }
#   #info-pane { flex:1; max-height: 70vh; overflow:auto; border:1px solid #ddd; padding:0.75rem; background:#fafafa; }
#   .toolbar { display:flex; gap:0.5rem; align-items:center; margin-bottom:0.5rem; flex-wrap: wrap; }
#   .toolbar input[type="text"] { flex:1; min-width: 220px; padding:0.5rem; border:1px solid #ccc; border-radius:8px; }
#   .toolbar button { padding:0.5rem 0.75rem; border:1px solid #ccc; border-radius:8px; background:#fff; cursor:pointer; }
#   .legend { display:flex; align-items:center; gap:.5rem; flex-wrap:wrap; font-size:13px; }
#   .legend .item { display:inline-flex; align-items:center; gap:.35rem; margin-right:.75rem; }
#   .legend .swatch { width:14px; height:14px; display:inline-block; border-radius:2px; border:1px solid rgba(0,0,0,.2); }
#   .node circle { cursor:grab; stroke:#333; stroke-width:0.17px; }
#   .node:active circle { cursor:grabbing; }
#   .node.selected circle { stroke:#111; stroke-width:1px; }
#   .dimmed { opacity:0.15; }
#   .label { font: 10px/1.2 system-ui, -apple-system, Segoe UI, Roboto, Ubuntu, sans-serif; pointer-events:none; }
#   .edge { stroke:#aaa; stroke-opacity:0.85; fill:none; }
#   .edge.highlight { stroke:#333; }
#   .edge-label { font-size: 9px; line-height: 1; fill:#555; }
#   .hidden { display:none !important; }

#   .sl-controls { margin: 0.25rem 0 0.5rem; display:flex; gap:0.5rem; }
#   #tabela-vertices .tabulator-cell, #tabela-arestas .tabulator-cell {
#     white-space: nowrap; text-overflow: ellipsis; overflow: hidden;
#   }
# </style>
# </head>
# <body>

# <h2>Rede de Coautoria entre PPGs (Fiocruz)</h2>
# <p>
#   Visualização interativa do grafo (nós = PPGs; arestas = colaborações). As cores dos nós são proporcionais ao número de <em>Conexões</em> (menor→maior), e a espessura das arestas ao <em>peso</em>.
# </p>

# <div class="viz-wrap">
#   <div style="flex:3; display:flex; flex-direction:column;">
#     <div class="toolbar">
#       <div class="legend" id="legend"></div>
#       <button id="btnToggleLabels" title="Mostrar/ocultar rótulos (vértices e arestas)">Ocultar rótulos</button>
#       <input id="searchBox" type="text" placeholder="Buscar PPG (nome/ID)..." />
#       <button id="btnSearch">Buscar</button>
#       <button id="btnClear">Limpar seleção</button>
#     </div>
#     <div id="viz-container">
#       <svg id="viz"></svg>
#     </div>
#   </div>
#   <div id="info-pane">
#     <h4>Informações do PPG</h4>
#     <p>Clique em um nó (ou use a busca) para ver detalhes aqui.</p>
#   </div>
# </div>

# <hr style="margin:1.25rem 0">

# <h3>Vértices</h3>
# <p>Tabela com atributos dos PPGs.</p>
# <div class="sl-controls">
#   <button id="vert-csv">⬇️ CSV</button>
#   <button id="vert-xlsx">⬇️ XLSX</button>
# </div>
# <div id="tabela-vertices"></div>

# <h3 style="margin-top:1.5rem;">Arestas (pares de PPGs)</h3>
# <p>Veja os pares que mais colaboram e quantos vizinhos em comum possuem.</p>
# <div class="sl-controls">
#   <button id="edge-csv">⬇️ CSV</button>
#   <button id="edge-xlsx">⬇️ XLSX</button>
# </div>
# <div id="tabela-arestas"></div>

# <script src="__D3_CDN__"></script>
# <script src="__TABULATOR_JS__"></script>
# <script src="__XLSX_JS__"></script>

# <script>
#   // ===== Dados embutidos (gerados no Python) =====
#   var GRAPH_DATA = __GRAPH_JSON__;
#   var DATA_VERT  = __VERTEX_TABLE_JSON__;
#   var DATA_EDGES = __EDGE_TABLE_JSON__;

#   // ===== Render básico (usa x,y do Python; sem simulação) =====
#   var svg = d3.select("#viz"),
#       g = svg.append("g"),
#       edgeLayer = g.append("g").attr("class","edges"),
#       nodeLayer = g.append("g").attr("class","nodes"),
#       labelLayer = g.append("g").attr("class","labels"),
#       edgeLabelLayer = g.append("g").attr("class","edge-labels");

#   var nodes = GRAPH_DATA.nodes.map(function(n){ return Object.assign({}, n); });
#   var links = GRAPH_DATA.links.map(function(e){ return Object.assign({}, e); });

#   var zoom = d3.zoom().scaleExtent([0.2, 8]).on("zoom", function(ev){ g.attr("transform", ev.transform); });
#   svg.call(zoom);

#   function getBox(){ return document.getElementById("viz-container").getBoundingClientRect(); }
#   var box = getBox(), width = box.width, height = box.height;
#   svg.attr("width", width).attr("height", height);

#   var weights = links.map(function(d){ return +d.weight || 0; });
#   var minW = d3.min(weights) || 0, maxW = d3.max(weights) || 1;
#   var edgeWidth   = d3.scaleLinear().domain([minW, maxW]).range([1.2, 10]).clamp(true);
#   var edgeOpacity = d3.scaleLinear().domain([minW, maxW]).range([0.56, 0.82]).clamp(true);

#   var edge = edgeLayer.selectAll("path")
#     .data(links)
#     .join("path")
#     .attr("class","edge")
#     .attr("id", function(_,i){ return "e"+i; })
#     .attr("stroke", function(d){ return d.color || "#aaa"; })
#     .attr("stroke-width", function(d){ return edgeWidth(+d.weight || 0); })
#     .attr("stroke-opacity", function(d){ return edgeOpacity(+d.weight || 0); })
#     .attr("fill","none");

#   var edgeLabels = edgeLabelLayer.selectAll("text")
#     .data(links).join("text")
#     .attr("class","edge-label")
#     .append("textPath")
#     .attr("href", function(_,i){ return "#e"+i; })
#     .attr("startOffset","50%")
#     .attr("text-anchor","middle")
#     .text(function(d){ return d.weight ? String(d.weight) : ""; });

#   var node = nodeLayer.selectAll("g.node")
#     .data(nodes, function(d){ return d.id; })
#     .join(function(enter){
#       var g = enter.append("g").attr("class","node");
#       g.append("circle")
#         .attr("r", function(d){ return +d.r || 8; })
#         .attr("fill", function(d){ return d.color || "#1f77b4"; });
#       return g;
#     });

#   var labels = labelLayer.selectAll("text")
#     .data(nodes, function(d){ return d.id; })
#     .join("text")
#     .attr("class","label")
#     .attr("text-anchor","middle")
#     .text(function(d){ return d.label; });

#   function updatePositions(){
#     edge.attr("d", function(d){
#       var sx = (typeof d.source === 'object') ? d.source.x : (nodes.find(n=>n.id===String(d.source))||{x:0}).x;
#       var sy = (typeof d.source === 'object') ? d.source.y : (nodes.find(n=>n.id===String(d.source))||{y:0}).y;
#       var tx = (typeof d.target === 'object') ? d.target.x : (nodes.find(n=>n.id===String(d.target))||{x:0}).x;
#       var ty = (typeof d.target === 'object') ? d.target.y : (nodes.find(n=>n.id===String(d.target))||{y:0}).y;
#       return "M"+sx+","+sy+" L"+tx+","+ty;
#     });
#     node.attr("transform", function(d){ return "translate("+d.x+", "+d.y+")"; });
#     labels.attr("x", function(d){ return d.x; }).attr("y", function(d){ return d.y - (d.r || 8) - 2; });
#   }
#   updatePositions();

#   node.call(d3.drag()
#     .on("start", function(ev, d){ d.fx = d.x; d.fy = d.y; })
#     .on("drag",  function(ev, d){ d.x = ev.x; d.y = ev.y; updatePositions(); })
#     .on("end",   function(ev, d){ d.fx = null; d.fy = null; })
#   );

#   var selectedId = null;
#   function neighborSet(id){
#     var set = new Set([id]);
#     links.forEach(function(e){
#       var s = (typeof e.source === 'object') ? e.source.id : String(e.source);
#       var t = (typeof e.target === 'object') ? e.target.id : String(e.target);
#       if (s === id) set.add(t);
#       if (t === id) set.add(s);
#     });
#     return set;
#   }

#   function renderSelection(){
#     if (!selectedId){
#       node.classed("selected", false);
#       node.classed("dimmed", false);
#       labels.classed("dimmed", false);
#       edge.classed("highlight", false).classed("dimmed", false);
#       edgeLabels.classed("dimmed", false);
#       return;
#     }
#     var keep = neighborSet(selectedId);
#     node.classed("selected", function(d){ return d.id === selectedId; })
#         .classed("dimmed", function(d){ return !keep.has(d.id); });
#     labels.classed("dimmed", function(d){ return !keep.has(d.id); });
#     edge.classed("highlight", function(d){
#           var s = (typeof d.source === 'object') ? d.source.id : String(d.source);
#           var t = (typeof d.target === 'object') ? d.target.id : String(d.target);
#           return (s === selectedId || t === selectedId);
#         })
#         .classed("dimmed", function(d){
#           var s = (typeof d.source === 'object') ? d.source.id : String(d.source);
#           var t = (typeof d.target === 'object') ? d.target.id : String(d.target);
#           return !(s === selectedId || t === selectedId);
#         });
#     edgeLabels.classed("dimmed", function(d){
#       var s = (typeof d.source === 'object') ? d.source.id : String(d.source);
#       var t = (typeof d.target === 'object') ? d.target.id : String(d.target);
#       return !(s === selectedId || t === selectedId);
#     });
#   }

# function fillInfo(d){
#     var pane = document.getElementById("info-pane");
#     // Mapeamento dos novos campos para exibição
#     var campos = {
#       "Nome": d.label,
#       "ID": d.id,
#       "Publicações Totais": d.total_publications,
#       "Score": d.score ? d.score.toFixed(4) : "",
#       "Centralidade Grau": d.centralidade_grau ? d.centralidade_grau.toFixed(5) : "",
#       "PageRank": d.pagerank ? d.pagerank.toFixed(5) : "",
#       "Hub Score": d.hub_score ? d.hub_score.toFixed(5) : ""
#     };
    
#     var html = "<h4>"+(d.label || d.id)+"</h4><ul>";
#     Object.keys(campos).forEach(function(k){
#       var v = campos[k];
#       if (v === undefined || v === null || v === "") return;
#       html += "<li><strong>"+k+":</strong> "+v+"</li>";
#     });
#     html += "</ul>";
    
#     // Opcional: Mostrar similares (node2vec) se houver
#     if(d.similares && d.similares !== "nan"){
#         try {
#             // O GEXF salva aspas duplas escapadas, o JS deve parsear
#             // Se vier como string Python, pode precisar de replace de aspas simples
#             var sims = JSON.parse(d.similares.replace(/'/g, '"')); 
#             if(sims.length > 0){
#                 html += "<br><strong>Similares (Node2Vec):</strong><ul style='font-size:0.9em'>";
#                 sims.slice(0, 5).forEach(function(s){
#                     html += "<li>" + s.nome + " (" + (s.similaridade*100).toFixed(1) + "%)</li>";
#                 });
#                 html += "</ul>";
#             }
#         } catch(e) {}
#     }

#     pane.innerHTML = html;
#   }

#   function selectNodeById(id){
#     var d = nodes.find(function(n){ return n.id === id; });
#     if (!d) return;
#     selectedId = d.id; fillInfo(d); renderSelection();
#     var t = d3.zoomTransform(svg.node());
#     var scale = Math.max(1.2, t.k);
#     var x = width/2 - d.x * scale;
#     var y = height/2 - d.y * scale;
#     svg.transition().duration(600).call(zoom.transform, d3.zoomIdentity.translate(x, y).scale(scale));
#   }
#   node.on("click", function(_, d){ selectNodeById(d.id); });

#   function norm(s){
#     return (s||"").toString().normalize("NFD").replace(/\\p{Diacritic}/gu,"").toLowerCase().trim();
#   }
#   function searchAndSelect(q){
#     var nq = norm(q); if (!nq) return;
#     var hit = nodes.find(function(n){ return norm(n.label) === nq || norm(n.id) === nq; });
#     if (!hit) hit = nodes.find(function(n){ return norm(n.label).indexOf(nq) >= 0; });
#     if (hit) selectNodeById(hit.id);
#   }
#   document.getElementById("btnSearch").addEventListener("click", function(){
#     searchAndSelect(document.getElementById("searchBox").value);
#   });
#   document.getElementById("searchBox").addEventListener("keydown", function(ev){
#     if (ev.key === "Enter") searchAndSelect(ev.target.value);
#   });
#   document.getElementById("btnClear").addEventListener("click", function(){
#     selectedId = null; renderSelection();
#     document.getElementById("info-pane").innerHTML =
#       "<h4>Informações do PPG</h4><p>Clique em um nó (ou use a busca) para ver detalhes aqui.</p>";
#   });

#   var labelsVisible = true;
#   var btnToggle = document.getElementById("btnToggleLabels");
#   function applyLabelsVisibility(){
#     labelLayer.classed("hidden", !labelsVisible);
#     d3.select(".edge-labels").classed("hidden", !labelsVisible);
#   }
#   btnToggle.addEventListener("click", function(){
#     labelsVisible = !labelsVisible;
#     btnToggle.textContent = labelsVisible ? "Ocultar rótulos" : "Mostrar rótulos";
#     applyLabelsVisibility();
#   });
#   applyLabelsVisibility();

#   // ===== Legenda (por faixas lineares de Conexões) =====
#   (function legend(){
#     // Alterado para ler 'total_publications' (ou a métrica que você escolheu no build_graph)
#     var vals = DATA_VERT.map(function(v){ return +v.total_publications || 0; });
#     var vmin = d3.min(vals) || 0, vmax = d3.max(vals) || 1;
#     var palette = __PALETTE_ARRAY__;
#     var steps = palette.length, legendData = [];
#     for (var i=0; i<steps; i++){
#       var a = vmin + (i/steps)*(vmax - vmin);
#       var b = vmin + ((i+1)/steps)*(vmax - vmin);
#       legendData.push({ color: palette[i], range: (Math.round(a*100)/100) + " – " + (Math.round(b*100)/100) });
#     }
#     var L = d3.select("#legend");
#     L.append("span").text("Cor por Conexões: ");
#     L.selectAll("span.item")
#       .data(legendData)
#       .join("span")
#       .attr("class","item")
#       .html(function(d){ return '<span class="swatch" style="background:'+d.color+'"></span>'+d.range; });
#   }());

#   function maybeFit(){
#     var xs = nodes.map(function(d){ return d.x; }), ys = nodes.map(function(d){ return d.y; });
#     var minX = Math.min.apply(null, xs), maxX = Math.max.apply(null, xs);
#     var minY = Math.min.apply(null, ys), maxY = Math.max.apply(null, ys);
#     var w = Math.max(1, maxX - minX), h = Math.max(1, maxY - minY);
#     var margin = 40;
#     var kx = (width  - margin*2) / w;
#     var ky = (height - margin*2) / h;
#     var scale = Math.max(0.2, Math.min(3, Math.min(kx, ky)));
#     var tx = (width  - scale*(minX + maxX)) / 2;
#     var ty = (height - scale*(minY + maxY)) / 2;
#     svg.transition().duration(600)
#       .call(zoom.transform, d3.zoomIdentity.translate(tx, ty).scale(scale));
#   }
#   maybeFit();

#   var ro = new ResizeObserver(function(){
#     var b = getBox();
#     width = b.width; height = b.height;
#     svg.attr("width", width).attr("height", height);
#     maybeFit();
#   });
#   ro.observe(document.getElementById("viz-container"));

#   // ===== Tabelas (Tabulator) =====
#   function heightFor(n){
#     if (n <= 12) return "auto";
#     if (n <= 60) return "40vh";
#     return "60vh";
#   }
#   var HEIGHT_VERT = heightFor(DATA_VERT.length);
#   var HEIGHT_EDG  = heightFor(DATA_EDGES.length);

#   const LANG_PT = {
#     "pt-br": {
#       "pagination": {
#         "first":"«","first_title":"Primeira",
#         "last":"»","last_title":"Última",
#         "prev":"‹","prev_title":"Anterior",
#         "next":"›","next_title":"Próxima"
#       },
#       "headerFilters": { "default":"⎯ Filtrar ⎯" }
#     }
#   };

#   function createTable(el, data, columns, initialSort, height){
#     const base = {
#       data,
#       layout: "fitColumns",
#       responsiveLayout: "collapse",
#       pagination: "local",
#       paginationSize: 50,
#       paginationSizeSelector: [10, 25, 50, 100, 200, 500],
#       movableColumns: true,
#       initialSort,
#       headerFilterLiveFilter: true,
#       columns,
#       langs: LANG_PT, locale: "pt-br",
#       tooltips: true
#     };
#     if (height !== "auto") base.height = height;
#     return new Tabulator(el, base);
#   }

#   (function VERTICES(){
#     const columns = [
#       { title:"ID", field:"id", headerFilter:"input", tooltip:true, width: 80 },
#       { title:"Nome", field:"nome", headerFilter:"input", widthGrow:2, tooltip:true },
#       { title:"Publicações", field:"total_publications", hozAlign:"right", sorter:"number", headerFilter:"input", tooltip:true },
#       { title:"Score", field:"score", hozAlign:"right", sorter:"number", headerFilter:"input", tooltip:true },
#       { title:"Centr. Grau", field:"centralidade_grau", hozAlign:"right", sorter:"number", headerFilter:"input", tooltip:true },
#       { title:"PageRank", field:"pagerank", hozAlign:"right", sorter:"number", headerFilter:"input", tooltip:true },
#       { title:"Cor", field:"cor", headerFilter:"input", tooltip:true, width: 60 }
#     ];
#     // Ordenação padrão sugerida: Score ou Publicações
#     const sort = [{ column:"total_publications", dir:"desc" }];
#     const t = createTable("#tabela-vertices", DATA_VERT, columns, sort, HEIGHT_VERT);
#     document.getElementById("vert-csv").onclick  = () => t.download("csv",  "vertices.csv", { bom:true });
#     document.getElementById("vert-xlsx").onclick = () => t.download("xlsx", "vertices.xlsx", { sheetName:"Vértices" });
#   })();

#   (function ARESTAS(){
#     const columns = [
#       { title:"Aresta", field:"aresta", headerFilter:"input", widthGrow:3, tooltip:true },
#       { title:"Número de colaborações", field:"colabs", hozAlign:"right", sorter:"number", headerFilter:"input", tooltip:true },
#       { title:"Nº de vizinhos em comum", field:"comuns", hozAlign:"right", sorter:"number", headerFilter:"input", tooltip:true }
#     ];
#     const sort = [{ column:"colabs", dir:"desc" }, { column:"comuns", dir:"desc" }];
#     const t = createTable("#tabela-arestas", DATA_EDGES, columns, sort, HEIGHT_EDG);
#     document.getElementById("edge-csv").onclick  = () => t.download("csv",  "arestas.csv", { bom:true });
#     document.getElementById("edge-xlsx").onclick = () => t.download("xlsx", "arestas.xlsx", { sheetName:"Arestas" });
#   })();
# </script>

# </body>
# </html>
# """
#     # Substituições finais
#     html_str = (html_str
#         .replace("__TABULATOR_CSS__", TABULATOR_CSS)
#         .replace("__D3_CDN__", D3_CDN)
#         .replace("__TABULATOR_JS__", TABULATOR_JS)
#         .replace("__XLSX_JS__", XLSX_JS)
#         .replace("__GRAPH_JSON__", graph_json_str)
#         .replace("__VERTEX_TABLE_JSON__", vertex_table_str)
#         .replace("__EDGE_TABLE_JSON__", edge_table_str)
#         .replace("__PALETTE_ARRAY__", "[" + ",".join(json.dumps(c) for c in PALETTE) + "]")
#     )
#     return html_str


In [50]:
# Caminho do seu arquivo .gexf
gexf_path = "src/new_graph.gexf"

# Defina manualmente os parâmetros (antes eles vinham de argparse)
seed = 42
k = 0.5
iterations = 400
scale = 0.7
node_radius = 8.0

# Nome do arquivo de saída
out_html = os.path.splitext(gexf_path)[0] + ".html"

print("Existe?", os.path.exists(gexf_path))
print("Tamanho (bytes):", os.path.getsize(gexf_path) if os.path.exists(gexf_path) else "NA")

# Execução
G = build_graph(
    gexf_path,
    seed=seed,
    k=k,
    iterations=iterations,
    scale=scale
)

data = graph_to_embeddable_json(G, node_radius_const=node_radius)

graph_json_str  = json.dumps(data["graph"], ensure_ascii=False)
vertex_table_str = json.dumps(data["vertex_table"], ensure_ascii=False)
edge_table_str   = json.dumps(data["edge_table"], ensure_ascii=False)

html_str = html_template(graph_json_str, vertex_table_str, edge_table_str)

with open(out_html, "w", encoding="utf-8") as f:
    f.write(html_str)

print(f"✅ Arquivo HTML gerado: {out_html}")


Existe? True
Tamanho (bytes): 18535410
✅ Arquivo HTML gerado: src/new_graph.html
